## dependencies installeren

In [2]:
%run ./__init__edwh__notebooks.ipynb

Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) 
[GCC 9.4.0]
 * postgresql://postgres@pgpool:5432/backend
Done.


## Tabellen en kolommen: 

In [3]:
for table in sorted(db.tables):
    print(f'# {table}')
    print(f'  ',', '.join(db[table].fields))
    print('')


# attachment
   id, platform, gid, attachment, filename, purpose, ts_uploaded, owner_gid, b2_uri

# click__event
   id, short_code, session_gid_hash, from_item_gid, ts

# click__url
   id, short_code, long_url

# comment
   id, platform, author, gid, concerning, in_response_to, body, ts_created, ts_last_change

# email_domain
   id, platform, gid, org_gid, domain, is_new_user_allowed

# event_stream
   id, gid, platform, user_gid, subject_gid, subject_type, ts, title, msg, doc, crud, session_token

# fav_list
   id, platform, gid, name, slug

# fav_list_member
   id, platform, gid, list_gid, user_gid, ts, list_role

# item
   id, platform, author, name, gid, thumbnail, short_description, ts_changed, tags, slug, alternatives, backgrounds, attachments, since_when, upto_when, video_urls

# mark
   id, platform, gid, user_gid, subject_gid, subject_type, list_gid, mark, name, ts

# organisation
   id, platform, gid, name, coc, street, number, city, tag_gid, lonlat

# property_bag
   id, gid

In [4]:
# selecteer 10 demo regels van items
# (nieuwe bovenaan, niet verwijderd, uit het SvS platform)
rows = db((db.item.author!=None) & (db.item.platform.contains('SvS'))).select(db.item.gid, db.item.name, limitby=(0,10), orderby=~db.item.ts_changed)
rows.as_csv().split('\r\n')

['item.gid,item.name',
 '00e0ec90-7ba5-4265-90f0-747b944963df,Uit de schatkist van Joke Middelbeek – Mopperschriftje',
 '521e38a1-441b-469f-bf10-58612dc172e5,Uit de schatkist van Joke Middelbeek – Wilskracht',
 'f5d2cdde-519c-4c4b-9b57-126b6de28a66,Uit de schatkist van Joke Middelbeek – Met vallen en opstaan',
 '8202bb06-39e6-4f0b-991d-90f2bd46f220,Uit de schatkist van Joke Middelbeek – Scholentour',
 '8ef0b366-1d56-43e5-9852-5e772e2ff30c,Uit de schatkist van Joke Middelbeek – Driftbuien',
 '6cfbf03e-5aca-4a28-9b66-2dca6afd332f,Kritisch leren denken tijdens het Museum Debat',
 '7090dd0e-0299-4aae-9a2d-467f61c338a0,De verbinding tussen informatica en de leefwereld van leerlingen',
 'e7b7db6c-fbf3-40f5-ae24-62536bd48963,Samen onze rekendidactiek verbeteren',
 '89d9ae56-ce82-419b-aaff-6bfa3713d4e6,Door Lezen de Wereld Ontmoeten',
 '12c166e8-672c-4f99-b449-7b9369e21abf,Kansengelijkheid vergroten: leerlingen worden eigenaar van hun ontwikkeling',
 '']

## SQL toegang krijgen middels 
zodat dit kan werken: 

In [5]:
%%sql
select * from item limit (2)

 * postgresql://postgres@pgpool:5432/backend
2 rows affected.


id,platform,author,name,gid,thumbnail,short_description,ts_changed,tags,slug,alternatives,backgrounds,attachments,since_when,upto_when,video_urls
6270,SvS,0630ddfb-be7a-47fe-91eb-3688026a5e58,Leerlijnen voor het speciaal onderwijs,d135365e-e8cc-4733-aaf3-12d2c6140488,f20cce5e-6e2b-4fd4-8fe6-24ba4f52897d,Op [SBO Merlijn](https://www.sbomerlijn.nl/) in Den Haag wordt gewerkt met leerlijnen voor lezen in het speciaal onderwijs. Bekijk een voorbeeld in de bijlage.,2020-01-24 12:26:55,|fbd1f8a3-4c2e-4dd0-8a59-d3578e17a766|4225d914-d4c8-4f89-85c5-11e26f853afa|ed64cc00-4457-4bb0-9fea-3e367f438297|8856b32d-dcfd-4a29-baa7-679888418938|8dbaad22-638d-46bd-8471-69181176fad6|a155acde-4ad8-4b8c-9693-a2eecef3c4c2|07abe8d6-8fa5-4730-80a7-88b735328924|c6231db3-8387-4b17-8c3c-4a45e0af8575|2e2b456b-3e22-4f26-a7a8-6213a521b1b3|b29162f2-bf80-4f24-aa9a-08b8ef3af4aa|03b6809f-4369-45a8-81b2-aa82592be856|43effd8b-a700-4e18-be2c-5a591793c5ca|,leerlijnen-voor-het-speciaal-onderwijs,None,||,|81a8b67b-5dce-4620-8e64-dd00470b1b3b|,2016-01-01,None,None
6828,SvS,a2f28371-72fe-4962-ab28-6c72624e3282,Test:8077136100629482,360c5d80-8719-455e-a8ff-1733dd3d523f,None,Description...,2021-04-26 16:00:29,||,test-8077136100629482,None,||,||,2021-01-01,None,None


of bijvoorbeeld 

In [6]:
%%sql
select to_char(ts,'YYYY-MM') as year_month,  count(*) as  items_created
from event_stream
where platform = 'SvS'
  and subject_type = 'item'
  and crud = 'c'
group by crud, subject_type,  to_char(ts,'YYYY-MM')
order by to_char(ts,'YYYY-MM') desc , subject_type, crud
limit 5;

 * postgresql://postgres@pgpool:5432/backend
5 rows affected.


year_month,items_created
2021-12,7
2021-11,5
2021-10,5
2021-08,2
2021-07,1


## Panda's gebruiken vanaf pydal resultaat

In [7]:
items = pd.read_csv(BytesIO(rows.as_csv().encode('utf-8')))

In [8]:
items

,item.gid,item.name
0,00e0ec90-7ba5-4265-90f0-747b944963df,Uit de schatkist van Joke Middelbeek – Moppers...
1,521e38a1-441b-469f-bf10-58612dc172e5,Uit de schatkist van Joke Middelbeek – Wilskracht
2,f5d2cdde-519c-4c4b-9b57-126b6de28a66,Uit de schatkist van Joke Middelbeek – Met val...
3,8202bb06-39e6-4f0b-991d-90f2bd46f220,Uit de schatkist van Joke Middelbeek – Scholen...
4,8ef0b366-1d56-43e5-9852-5e772e2ff30c,Uit de schatkist van Joke Middelbeek – Driftbuien
5,6cfbf03e-5aca-4a28-9b66-2dca6afd332f,Kritisch leren denken tijdens het Museum Debat
6,7090dd0e-0299-4aae-9a2d-467f61c338a0,De verbinding tussen informatica en de leefwer...
7,e7b7db6c-fbf3-40f5-ae24-62536bd48963,Samen onze rekendidactiek verbeteren
8,89d9ae56-ce82-419b-aaff-6bfa3713d4e6,Door Lezen de Wereld Ontmoeten
9,12c166e8-672c-4f99-b449-7b9369e21abf,Kansengelijkheid vergroten: leerlingen worden ...


## Gebruik van libraries icm jupyterlab 

In [9]:
handig.zeer_handig

True

In [10]:
# als de library gewijzigd is en je wil niet de hele kernel herstarten
# maar wel de nieuwe library gebruiken, kun je dit doen: 
from importlib import reload 
handig = reload(handig)

## Tags


In [11]:
# make available as regular variables in the python_environment
Tag.create_Tnamed_metatag_variables(locals())

# show these items
for k,v in locals().copy().items():
    if k.startswith('T') and isinstance(v,Tag):
        print(f'{k:>20} : {v.gid}')

      TOrganisations : e5b80b90-1851-42d9-aebc-87327e87ed09
               TItem : fc8a7d36-e23e-4aba-8ee9-f5269ecc8dfc
TUser_generated_tags : 38f7b956-6315-4676-8076-8aeef258563e
             TSystem : 19682a99-50a3-4fc0-bb67-e0f6eff5da55
             TFilter : 8b78e33d-c4ab-442e-aaf6-411fda089b03
   TIsuserselectable : dcaea9b5-879e-43f7-8860-67579e10166e
         TFilterbalk : c05e9cea-76c0-4b48-8f91-4cee4ff7f0ab


In [12]:
# find all filters :
filter_tags = Tag.find_by_meta_tag(TFilter)
dict([(r.name, Tag.find_children(Tag(r))) for r in filter_tags])

{'Doelgroep': <Rows (22)>,
 'Omvang': <Rows (8)>,
 'Dimensie': <Rows (9)>,
 'Offline aanbod': <Rows (6)>,
 'Geplaatst door': <Rows (19)>,
 'Media': <Rows (3)>,
 'Sector': <Rows (5)>}

# Zoek fouten in de Tag structuur

Een parent property en een child property hoeven niet in sync met elkaar te zijn. Dus door het vergelijken van de parents van elke child, en de children van elke parent is na te gaan waar deze niet langer kloppend zijn.  
<font color="red"> **⚠** Fix deze issues </font>

In [13]:
# handig.find_out_of_sync_parent_child_relationships(Tag)

In [14]:
handig.human_readable_list_of_tagged_with(Tag, TFilter)

## Doelgroep

 * 0 t/m 3 jaar (KDV/VVE/peuterschool) <font color="darkgrey">[ Item, isUserSelectable ]</font>

 * 4 t/m 7 jaar (onderbouw PO) <font color="darkgrey">[ Item, isUserSelectable ]</font>

 * 7 t/m 10 jaar (middenbouw PO) <font color="darkgrey">[ Item, isUserSelectable ]</font>

 * 10 t/m 12 jaar (bovenbouw PO) <font color="darkgrey">[ Item, isUserSelectable ]</font>

...

 * Kinderdagverblijf (KDV) / Voor- en Vroegschoolse Educatie (VVE) / Peuterschool <font color="darkgrey">[ Item, isUserSelectable ]</font>

 * Primair onderwijs (po) <font color="darkgrey">[ Item, isUserSelectable ]</font>

 * Buitenschoolse opvang (bso) <font color="darkgrey">[ Item, isUserSelectable ]</font>

 * Voortgezet onderwijs (vo) <font color="darkgrey">[ Item, isUserSelectable ]</font>

 * Middelbaar beroeps onderwijs (MBO) <font color="darkgrey">[ Item, isUserSelectable ]</font>

## Walk door de boomstructuur van Tags 

In [15]:
TSystem.walk(_print)

 System ['System']
    UIRelated ['System']
       Filter ['System']
       User generated tags ['System']
           *** hidden ***
       Filterbalk ['System', 'Filterbalk']
          Coronavirus ['Item']
          Professionalisering ['Item']
          Anders organiseren ['Item']
             Coronavirus ['Item']
          Differentiatie ['Item', 'isUserSelectable']
          School en samenleving ['Item', 'isUserSelectable']
          Het volgen van leerlingen ['Item', 'isUserSelectable']
          21ste Eeuwse vaardigheden ['Item']
       Stickers ['System']
          SlimFit onderwijs ['User generated tags', 'Item']
          Voortgezet Leren []
          Slimmerlerenmetict.nl -> PO Raad []
          Schoolleiders voor de Toekomst []
          Kennisnet podcasts []
          Lesopafstand.nl ['System']
          Meesterwerk Podcast ['System']
          test-sticker ['System']
          School aan Zet ['System']
          Leraren met Lef []
          Leerling2020 ['User generated t

In [16]:
# of gebruikt de specifieke richting (via een method van Tag)
Tag('System').walk(_print, direction='children_by_parental_reference')

 System ['System']
    Organisatie ['Item', 'User generated tags']
    UIRelated ['System']
       Stickers ['System']
          Schoolkracht ['System']
          NRO Kennisrotonde []
          Voortgezet Leren []
          TOM (Team Onderwijs op Maat) []
          Slimmerlerenmetict.nl -> PO Raad []
          LOF (Lerarenontwikkelfonds) []
          Leraren met Lef []
          Leraar24.nl []
          OnderwijsInc []
          leraar.nl []
          Kennisnet podcasts []
          DNA van de leraar []
          voordeklas.com []
          Schoolleiders voor de Toekomst []
          VO Raad []
          PACT voor Kindcentra []
          stichting leerKRACHT []
          vernieuwenderwijs []
          Facebook groepen []
          Reformatorisch Dagblad []
          Eenvandaag []
          HUPP ['System']
          SlimFit onderwijs ['User generated tags', 'Item']
          Lesopafstand.nl ['System']
          Leren met de Stad ['System']
          Meesterwerk Podcast ['System']
      